# Projet maison n° 4

In [1]:
# imports
import requests
from bs4 import BeautifulSoup

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [3]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
import sys
import re

def get_prices_from_url(url):
    prices = {}
    r = requests.get(url).content.decode('utf-8')
    soup = BeautifulSoup(r)
    num_page = str(int(re.findall(r'-?\d+\.?\d*', url)[0][0]) + 1)
    for div in soup.findAll('div', class_="pure-u-1 pure-u-md-1-" + num_page) :
        div_desc = div.descendants
        for d in div_desc :

            if d.name == 'li' and 'database' in d.text :
                text = d.text
                ndatabase_1 = str(re.findall(r'-?\d+\.?\d*', text)[0])
                ndatabase_2 = str(re.findall("[A-Za-z]+",text)[0])

            if d.name == 'li' and ('GB' in d.text or 'TB' in d.text) :
                text = d.text
                
                stockage_1 = str(re.findall(r'-?\d+\.?\d*', text)[0])
                stockage_2 = str(re.findall("[A-Za-z]+",text)[0])
            
            if d.name == 'h2' :
                categ = d.text

            if d.name == 'span' : 
                span_desc = d.descendants
                for s in span_desc :
                    if '$' in s.text :
                        prix = s.text.replace(' ','')
                        prix = prix.replace('\n','')

        prices[categ] = {'price' : prix, 'storage' : stockage_1 + stockage_2, 'databases' : int(ndatabase_1) }
        
    return prices

get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [4]:
# partie B-1
def extract_beer_infos(url):
    beer_infos = {
        'Nom': None,
        'Style': None,
        'Contenu': None,
        'Degré d’alcool': None,
        'Origine': None,
        'Brasseur': None
    }
    r = requests.get(url).content.decode('utf-8')
    soup = BeautifulSoup(r)
    rows = soup.find('dl', class_='row')

    style = rows.find(text='Style').find_next().text
    alcool = rows.find(text='Degré d’alcool').find_next().text
    contenu = rows.find(text='Contenu').find_next().text
    origine = rows.find(text= 'Origine').find_next().text
    nom = soup.find('h1').text
    brasseur = rows.find(text= 'Brasseur').find_next().text
    
    beer_infos['Nom'] = nom
    beer_infos['Style'] = style.replace('\n','')
    beer_infos['Contenu'] = int(re.findall(r'-?\d+\.?\d*',contenu)[0])
    beer_infos['Degré d’alcool'] = float(alcool.replace('%','').replace(',','.'))
    beer_infos['Origine'] = origine.replace('\n','').replace('\r','').replace(' ','')
    beer_infos['Brasseur'] = brasseur.replace('\n','').replace('\r','').replace(' ','')



    return beer_infos


extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/la-cristal-ipa-du-mont-blanc.33')
#extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/pelforth-blonde-fut-5l')


{'Nom': 'La Cristal IPA du Mont Blanc',
 'Style': 'IPA',
 'Contenu': 33,
 'Degré d’alcool': 4.7,
 'Origine': 'France',
 'Brasseur': 'Mont-Blanc'}

In [5]:
# partie B-2


URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
import json
from multiprocessing import Pool
from time import time

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    pages = requests.get(url).content

    pages = json.loads(pages)
    for page in pages['items'] :
        beer_pages.append(str('https://www.beerwulf.com' + page['contentReference']))
    
    # Sequential version (slow):
    beers = []
    for beer_page in beer_pages :
        beers.append(extract_beer_infos(beer_page))
        
    
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)




[{'Nom': 'La Cristal IPA du Mont Blanc',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Mont-Blanc'},
 {'Nom': 'Ninkasi Pale Ale',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'NinkasiFabriques'},
 {'Nom': 'Desperados',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Desperados'},
 {'Nom': 'Pietra',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'BrasseriePietra'}]

In [6]:
# Facultatif
    

from multiprocessing import Pool
from time import time

URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
import json
from multiprocessing import Pool
from time import time

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    pages = requests.get(url).content

    pages = json.loads(pages)
    for page in pages['items'] :
        beer_pages.append(str('https://www.beerwulf.com' + page['contentReference']))
    
    # Parallel version (faster):
    beers = []
    
    if __name__ == "__main__":
       pool = Pool()
       pool.map(extract_beer_infos, beer_pages)
       pool.close()
       
    
    
    return beers

#extract_beer_list_infos(URL_BEERLIST_FRANCE)

In [6]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.941s

OK
